In [1]:
!pip install trl --upgrade
%pip install datasets==2.16.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 6.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.0
    Uninstalling fsspec-2024.6.0:
      Successfully uninstalled fsspec-2024.6.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfu

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification

In [3]:
import torch
from torch.utils.data import Dataset

from torch.utils.data import DataLoader

In [4]:
device = 'cuda'

In [5]:
PATH_SFT = '/kaggle/input/sft-opt/sft_opt/'
PATH_RM = '/kaggle/input/rm-opt-level2/RM_opt_2/'

tokenizer = AutoTokenizer.from_pretrained(PATH_SFT, local_files_only=True, padding_side='left')

#ref_model = AutoModelForCausalLM.from_pretrained(PATH_SFT, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(PATH_SFT, local_files_only=True)
reward_model = AutoModelForSequenceClassification.from_pretrained(PATH_RM, local_files_only=True)

In [6]:
model.to('cuda')
reward_model.to('cuda')

OPTForSequenceClassification(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Line

In [7]:
import datasets
from datasets import load_dataset

In [8]:
ds = load_dataset("nvidia/HelpSteer2")['train']

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [9]:
query = set(ds['prompt'])
tokenized_query = tokenizer(list(query)[:1000], return_tensors="pt", padding=True, truncation=True, max_length = 256)
input_ids = tokenized_query["input_ids"]
attention_mask = tokenized_query["attention_mask"]

In [10]:
class TextDataset(Dataset):
    def __init__(self, input_ids, attention_mask):
        self.input_ids = input_ids
        self.attention_mask = attention_mask

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx]
        }

dataset = TextDataset(input_ids, attention_mask)

In [11]:
dataloader = DataLoader(
    dataset,
    batch_size=4,  
    shuffle=True  
)

In [12]:
class r_ma:
    ma = 0

baseline = r_ma()

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [14]:
cnt = 0

class_indices = torch.arange(1, 11).float()  # Shape: (num_classes,)
class_indices = class_indices.to(device)

for batch in dataloader:
    cnt += 1
        
    batch_i = batch['input_ids']
    batch_m = batch['attention_mask']
    batch_i = batch_i.to(device)
    batch_m = batch_m.to(device)
    
    seq = model.generate(
                input_ids=batch_i,
                attention_mask=batch_m,
                max_length=512,  # Adjust as needed
                return_dict_in_generate=True,
                output_scores=True,
            )['sequences']
    
    output_m = (seq!=1).long()
    
    model.train()
    
    logits = model(seq)['logits']
    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)

    log_probs = torch.where(torch.isfinite(log_probs), log_probs, torch.zeros_like(log_probs))
    
    log_probs = log_probs[:, batch_i.shape[1]:, :]
    response = seq[:, batch_i.shape[1]:]

    actions = torch.gather(log_probs, dim=2, index=response.unsqueeze(-1))
    actions = actions.squeeze(-1)  
    
    with torch.no_grad():
        reward_scores = reward_model(input_ids = seq, attention_mask = output_m).logits
        reward_scores = reward_scores.softmax(dim=-1)
        reward_scores = torch.matmul(reward_scores, class_indices)
        reward_copy = reward_scores.sum()
        reward_scores -= baseline.ma
        reward_scores = reward_scores.unsqueeze(-1)
        
    loss = -(actions*reward_scores).sum(1).mean() # torch.Size([256, 4])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if cnt%20==0:
        print(cnt, loss.item(), reward_scores.mean().item())

    baseline.ma += reward_copy/10161

20 16.523305892944336 6.296265125274658
40 0.2507750689983368 6.26449728012085
60 12.530776977539062 6.327113628387451
80 18.63352394104004 6.048141002655029
100 14.845863342285156 5.593765735626221
120 4.410995960235596 5.741855144500732
140 9.32431411743164 5.817492485046387
160 7.609306335449219 6.278526306152344
180 10.802900314331055 6.149106025695801
200 3.2992305755615234 5.4473490715026855
220 27.357440948486328 6.006207466125488
240 0.5652889013290405 5.824312210083008


In [15]:
model.save_pretrained('REINFORCE_opt_2')